# Sudoku
Solve a sudoku puzzle.

In [ ]:
# To find the next unfilled cell in the board and return its position.
# The unfilled cells are denoted by a 0.
def next_Cell_to_Fill(grid) :
    for x in range(9) :
        for y in range(9) :
            if grid[x][y] == 0 :
                return(x, y)
    return(-1, -1)

# To validate the grid i.e if all the rules of sudoku are being maintained or not.
def validate_Grid(grid, x, y, e) :
    f = True
    for i in range(9) :
        if grid[x][i]==e :
            f = False
            break
    if f == True :
        # There is no element e in the horizontal row.
        g = True
        for i in range(9) :
            if grid[i][y]==e :
                g = False
                break
        if g == True:
            # There is no element e in the vertical column.
            startX = 3*(x//3)
            startY = 3*(y//3)
            for i in range(startX, startX+3) :
                for j in range(startY, startY+3) :
                    if grid[i][j] == e :
                        return(False)
            # There is no element e in the 3 X 3 grid in which the number e is to be placed.
            return(True)
    return(False)

# Main Function to solve the puzzle.
def solve_Sudoku(grid) :
    x, y = next_Cell_to_Fill(grid)
    if x == -1 or y == -1 :
        return(True)
    global backtrack
    for e in range(1, 10) :
        if validate_Grid(grid, x, y, e) :
            # We can place the number e in the x, y position of the grid.
            grid[x][y] = e
            if solve_Sudoku(grid) :
                return(True)
            backtrack += 1
            grid[x][y] = 0
    return(False)

# To print the final sudoku puzzle.
def print_Sudoku(grid) :
    global backtrack
    print('The solution of the sudoku is as follows :-')
    for x in range(9) :
        if x%3 == 0 :
            print()
        for y in range(9) :
            if y%3 == 0 :
                print('\t', end='')
            print(grid[x][y], end=' ')
            if y%8 == 0  and y!=0:
                print()
    print()
    print('The number of backtracks that has occured :- ', backtrack)

# To input the partial sudoku puzzle.
def enter_Sudoku(grid) :
    print('Enter the grid row by row denoting all the empty slots with a zero.')
    for i in range(9) :
        grid.append(list(map(int, input())))

grid = []
backtrack = 0
enter_Sudoku(grid)
solve_Sudoku(grid)
print_Sudoku(grid)

## Efficient Code with Implications

In [ ]:
# To find the next unfilled cell in the board and return its position.
# The unfilled cells are denoted by a 0.
def next_Cell_to_Fill(grid) :
    for x in range(9) :
        for y in range(9) :
            if grid[x][y] == 0 :
                return(x, y)
    return(-1, -1)

# To validate the grid i.e if all the rules of sudoku are being maintained or not.
def validate_Grid(grid, x, y, e) :
    f = True
    for i in range(9) :
        if grid[x][i]==e :
            f = False
            break
    if f == True :
        # There is no element e in the horizontal row.
        g = True
        for i in range(9) :
            if grid[i][y]==e :
                g = False
                break
        if g == True:
            # There is no element e in the vertical column.
            startX = 3*(x//3)
            startY = 3*(y//3)
            for i in range(startX, startX+3) :
                for j in range(startY, startY+3) :
                    if grid[i][j] == e :
                        return(False)
            # There is no element e in the 3 X 3 grid in which the number e is to be placed.
            return(True)
    return(False)

# The 9 various sectors of a sudoku grid, each list entry is the startX, endX, startY, startY of each sub-grid.
sectors = [ [0, 3, 0, 3], [3, 6, 0, 3], [6, 9, 0, 3],
            [0, 3, 3, 6], [3, 6, 3, 6], [6, 9, 3, 6],
            [0, 3, 6, 9], [3, 6, 6, 9], [6, 9, 6, 9] ]

# To make some implications based on the current choice of piece.
def make_Implications(grid, x, y, e) :
    global sectors
    grid[x][y] = e
    impl = [(x, y, e)]
    for k in range(len(sectors)) :
        totset = {1, 2, 3, 4, 5, 6, 7, 8, 9}
        # To remove all the already present elements of the sub-grid from the set of all possible combinations.
        for i in range(sectors[k][0], sectors[k][1]) :
            for j in range(sectors[k][2], sectors[k][3]) :
                if grid[i][j] != 0 :
                    totset.remove(grid[i][j])
        num = 0
        for i in range(sectors[k][0], sectors[k][1]) :
            for j in range(sectors[k][2], sectors[k][3]) :
                if grid[i][j] == 0 :
                    # An empty location in the sub-grid.
                    vset = totset.copy()                 # To store all the possible numbers for that particular empty location of the sub-grid under consideration.
                    for m in range(9) :
                        # To remove all the already present elements of the row from the set.
                        if grid[i][m] != 0 and grid[i][m] in vset :
                            vset.remove(grid[i][m])
                        # To remove all the already present elements of the column from the set.
                        if grid[m][j] != 0 and grid[m][j] in vset :
                            vset.remove(grid[m][j])
                    if len(vset) == 1 :
                        # As there is only one possibility we can now make an implication.
                        val = vset.pop()
                        if validate_Grid(grid, i, j, val) :
                            # The implication made has been validated for the present board condition.
                            grid[i][j] = val
                            impl.append((i, j, val))
                    num += 1
    return(impl)

# To undo all the implications that have been made.
def undo_Implications(grid, impl) :
    for implication in impl :
        grid[implication[0]][implication[1]] = 0

# Main Function to solve the puzzle.
def solve_Sudoku_Opt(grid) :
    x, y = next_Cell_to_Fill(grid)
    if x == -1 or y == -1 :
        return(True)
    global backtrack
    for e in range(1, 10) :
        if validate_Grid(grid, x, y, e) :
            # We can place the number e in the x, y position of the grid.
            impl = make_Implications(grid, x, y, e)
            if solve_Sudoku_Opt(grid) :
                return(True)
            backtrack += 1
            undo_Implications(grid, impl)
    return(False)

# To print the final sudoku puzzle.
def print_Sudoku(grid) :
    global backtrack
    print('The solution of the sudoku is as follows :-')
    for x in range(9) :
        if x%3 == 0 :
            print()
        for y in range(9) :
            if y%3 == 0 :
                print('\t', end='')
            print(grid[x][y], end=' ')
            if y%8 == 0  and y!=0:
                print()
    print()
    print('The number of backtracks that has occured :- ', backtrack)

# To input the partial sudoku puzzle.
def enter_Sudoku(grid) :
    print('Enter the grid row by row denoting all the empty slots with a zero.')
    for i in range(9) :
        grid.append(list(map(int, input())))

grid = []
backtrack = 0
enter_Sudoku(grid)
solve_Sudoku_Opt(grid)
print_Sudoku(grid)

## More Efficient Algorithm using a process of Multiple Implications
Every time we make an implication we proceed making more than one implication until we cannot make an implication.

In [ ]:
# To find the next unfilled cell in the board and return its position.
# The unfilled cells are denoted by a 0.
def next_Cell_to_Fill(grid) :
    for x in range(9) :
        for y in range(9) :
            if grid[x][y] == 0 :
                return(x, y)
    return(-1, -1)

# To validate the grid i.e if all the rules of sudoku are being maintained or not.
def validate_Grid(grid, x, y, e) :
    f = True
    for i in range(9) :
        if grid[x][i]==e :
            f = False
            break
    if f == True :
        # There is no element e in the horizontal row.
        g = True
        for i in range(9) :
            if grid[i][y]==e :
                g = False
                break
        if g == True:
            # There is no element e in the vertical column.
            startX = 3*(x//3)
            startY = 3*(y//3)
            for i in range(startX, startX+3) :
                for j in range(startY, startY+3) :
                    if grid[i][j] == e :
                        return(False)
            # There is no element e in the 3 X 3 grid in which the number e is to be placed.
            return(True)
    return(False)

# The 9 various sectors of a sudoku grid, each list entry is the startX, endX, startY, startY of each sub-grid.
sectors = [ [0, 3, 0, 3], [3, 6, 0, 3], [6, 9, 0, 3],
            [0, 3, 3, 6], [3, 6, 3, 6], [6, 9, 3, 6],
            [0, 3, 6, 9], [3, 6, 6, 9], [6, 9, 6, 9] ]

# To make some implications based on the current choice of piece.
def make_Implications(grid, x, y, e) :
    global done
    if done == False :
        return([])
    global sectors
    grid[x][y] = e
    impl = [(x, y, e)]
    for k in range(len(sectors)) :
        totset = {1, 2, 3, 4, 5, 6, 7, 8, 9}
        # To remove all the already present elements of the sub-grid from the set of all possible combinations.
        for i in range(sectors[k][0], sectors[k][1]) :
            for j in range(sectors[k][2], sectors[k][3]) :
                if grid[i][j] != 0 :
                    totset.remove(grid[i][j])
        num = 0
        for i in range(sectors[k][0], sectors[k][1]) :
            for j in range(sectors[k][2], sectors[k][3]) :
                if grid[i][j] == 0 :
                    # An empty location in the sub-grid.
                    vset = totset.copy()                 # To store all the possible numbers for that particular empty location of the sub-grid under consideration.
                    for m in range(9) :
                        # To remove all the already present elements of the row from the set.
                        if grid[i][m] != 0 and grid[i][m] in vset :
                            vset.remove(grid[i][m])
                        # To remove all the already present elements of the column from the set.
                        if grid[m][j] != 0 and grid[m][j] in vset :
                            vset.remove(grid[m][j])
                    if len(vset) == 1 :
                        # As there is only one possibility we can now make an implication.
                        val = vset.pop()
                        if validate_Grid(grid, i, j, val) :
                            # The implication made has been validated for the present board condition.
                            grid[i][j] = val
                            #impl.append((i, j, val))
                            newimpl = make_Implications(grid, i, j, val)
                            if len(newimpl) == 0 :
                                done = False
                            else :
                                impl += newimpl
                    num += 1
    return(impl)

# To undo all the implications that have been made.
def undo_Implications(grid, impl) :
    for implication in impl :
        grid[implication[0]][implication[1]] = 0

# Main Function to solve the puzzle.
def solve_Sudoku_Opt(grid) :
    x, y = next_Cell_to_Fill(grid)
    if x == -1 or y == -1 :
        return(True)
    global backtrack
    for e in range(1, 10) :
        if validate_Grid(grid, x, y, e) :
            # We can place the number e in the x, y position of the grid.
            impl = make_Implications(grid, x, y, e)
            if solve_Sudoku_Opt(grid) :
                return(True)
            backtrack += 1
            undo_Implications(grid, impl)
    return(False)

# To print the final sudoku puzzle.
def print_Sudoku(grid) :
    global backtrack
    print('The solution of the sudoku is as follows :-')
    for x in range(9) :
        if x%3 == 0 :
            print()
        for y in range(9) :
            if y%3 == 0 :
                print('\t', end='')
            print(grid[x][y], end=' ')
            if y%8 == 0  and y!=0:
                print()
    print()
    print('The number of backtracks that has occured :- ', backtrack)

# To input the partial sudoku puzzle.
def enter_Sudoku(grid) :
    print('Enter the grid row by row denoting all the empty slots with a zero.')
    for i in range(9) :
        grid.append(list(map(int, input())))

grid = []
backtrack = 0
enter_Sudoku(grid)
done = True
solve_Sudoku_Opt(grid)
print_Sudoku(grid)

## Most Efficient Algorithm using an Optimized process of Multiple Implications

In [ ]:
# To find the next unfilled cell in the board and return its position.
# The unfilled cells are denoted by a 0.
def next_Cell_to_Fill(grid) :
    for x in range(9) :
        for y in range(9) :
            if grid[x][y] == 0 :
                return(x, y)
    return(-1, -1)

# To validate the grid i.e if all the rules of sudoku are being maintained or not.
def validate_Grid(grid, x, y, e) :
    f = True
    for i in range(9) :
        if grid[x][i]==e :
            f = False
            break
    if f == True :
        # There is no element e in the horizontal row.
        g = True
        for i in range(9) :
            if grid[i][y]==e :
                g = False
                break
        if g == True:
            # There is no element e in the vertical column.
            startX = 3*(x//3)
            startY = 3*(y//3)
            for i in range(startX, startX+3) :
                for j in range(startY, startY+3) :
                    if grid[i][j] == e :
                        return(False)
            # There is no element e in the 3 X 3 grid in which the number e is to be placed.
            return(True)
    return(False)

# The 9 various sectors of a sudoku grid, each list entry is the startX, endX, startY, startY of each sub-grid.
sectors = [ [0, 3, 0, 3], [3, 6, 0, 3], [6, 9, 0, 3],
            [0, 3, 3, 6], [3, 6, 3, 6], [6, 9, 3, 6],
            [0, 3, 6, 9], [3, 6, 6, 9], [6, 9, 6, 9] ]

# To make some implications based on the current choice of piece.
def make_Implications(grid, x, y, e) :
    global sectors
    done = False
    grid[x][y] = e
    impl = [(x, y, e)]
    while not done :
        done = True
        for k in range(len(sectors)) :
            totset = {1, 2, 3, 4, 5, 6, 7, 8, 9}
            # To remove all the already present elements of the sub-grid from the set of all possible combinations.
            for i in range(sectors[k][0], sectors[k][1]) :
                for j in range(sectors[k][2], sectors[k][3]) :
                    if grid[i][j] != 0 :
                        totset.remove(grid[i][j])
            num = 0
            for i in range(sectors[k][0], sectors[k][1]) :
                for j in range(sectors[k][2], sectors[k][3]) :
                    if grid[i][j] == 0 :
                        # An empty location in the sub-grid.
                        vset = totset.copy()
                # To store all the possible numbers for that particular empty location of the sub-grid under consideration.
                        for m in range(9) :
                            # To remove all the already present elements of the row from the set.
                            if grid[i][m] != 0 and grid[i][m] in vset :
                                vset.remove(grid[i][m])
                            # To remove all the already present elements of the column from the set.
                            if grid[m][j] != 0 and grid[m][j] in vset :
                                vset.remove(grid[m][j])
                        if len(vset) == 1 :
                            # As there is only one possibility we can now make an implication.
                            val = vset.pop()
                            if validate_Grid(grid, i, j, val) :
                                # The implication made has been validated for the present board condition.
                                grid[i][j] = val
                                impl.append((i, j, val))
                                done = False
                        num += 1
    return(impl)

# To undo all the implications that have been made.
def undo_Implications(grid, impl) :
    for implication in impl :
        grid[implication[0]][implication[1]] = 0

# Main Function to solve the puzzle.
def solve_Sudoku_Opt(grid) :
    x, y = next_Cell_to_Fill(grid)
    if x == -1 or y == -1 :
        return(True)
    global backtrack
    for e in range(1, 10) :
        if validate_Grid(grid, x, y, e) :
            # We can place the number e in the x, y position of the grid.
            impl = make_Implications(grid, x, y, e)
            if solve_Sudoku_Opt(grid) :
                return(True)
            backtrack += 1
            undo_Implications(grid, impl)
    return(False)

# To print the final sudoku puzzle.
def print_Sudoku(grid) :
    global backtrack
    print('The solution of the sudoku is as follows :-')
    for x in range(9) :
        if x%3 == 0 :
            print()
        for y in range(9) :
            if y%3 == 0 :
                print('\t', end='')
            print(grid[x][y], end=' ')
            if y%8 == 0  and y!=0:
                print()
    print()
    print('The number of backtracks that has occured :- ', backtrack)

# To input the partial sudoku puzzle.
def enter_Sudoku(grid) :
    print('Enter the grid row by row denoting all the empty slots with a zero.')
    for i in range(9) :
        grid.append(list(map(int, input())))

grid = []
backtrack = 0
enter_Sudoku(grid)
solve_Sudoku_Opt(grid)
print_Sudoku(grid)

## Sample Sudoku puzzles

In [ ]:
input = [[5,1,7,6,0,0,0,3,4],
         [2,8,9,0,0,4,0,0,0],
         [3,4,6,2,0,5,0,9,0],
         [6,0,2,0,0,0,0,1,0],
         [0,3,8,0,0,6,0,4,7],
         [0,0,0,0,0,0,0,0,0],
         [0,9,0,0,0,0,0,7,8],
         [7,0,3,4,0,0,5,6,0],
         [0,0,0,0,0,0,0,0,0]]

inp2  = [[5,1,7,6,0,0,0,3,4],
         [0,8,9,0,0,4,0,0,0],
         [3,0,6,2,0,5,0,9,0],
         [6,0,0,0,0,0,0,1,0],
         [0,3,0,0,0,6,0,4,7],
         [0,0,0,0,0,0,0,0,0],
         [0,9,0,0,0,0,0,7,8],
         [7,0,3,4,0,0,5,6,0],
         [0,0,0,0,0,0,0,0,0]]

inpd  = [[1,0,5,7,0,2,6,3,8],
         [2,0,0,0,0,6,0,0,5],
         [0,6,3,8,4,0,2,1,0],
         [0,5,9,2,0,1,3,8,0],
         [0,0,2,0,5,8,0,0,9],
         [7,1,0,0,3,0,5,0,2],
         [0,0,4,5,6,0,7,2,0],
         [5,0,0,0,0,4,0,6,3],
         [3,2,6,1,0,7,0,0,4]]

hard  = [[8,5,0,0,0,2,4,0,0],
         [7,2,0,0,0,0,0,0,9],
         [0,0,4,0,0,0,0,0,0],
         [0,0,0,1,0,7,0,0,2],
         [3,0,5,0,0,0,9,0,0],
         [0,4,0,0,0,0,0,0,0],
         [0,0,0,0,8,0,0,7,0],
         [0,1,7,0,0,0,0,0,0],
         [0,0,0,0,3,6,0,4,0]]

diff  = [[0,0,5,3,0,0,0,0,0],
         [8,0,0,0,0,0,0,2,0],
         [0,7,0,0,1,0,5,0,0],
         [4,0,0,0,0,5,3,0,0],
         [0,1,0,0,7,0,0,0,6],
         [0,0,3,2,0,0,0,8,0],
         [0,6,0,5,0,0,0,0,9],
         [0,0,4,0,0,0,0,3,0],
         [0,0,0,0,0,9,7,0,0]]